In [10]:
import torch
import random

### 生成数据集与小批量样本

In [11]:
dim_features = 8
true_w = torch.normal(0, 4, (dim_features, ))
true_b = torch.normal(4, 1, (1, ))
true_w, true_b

(tensor([ 0.5277,  1.7805, -4.3920, -0.2680,  0.5007,  0.3966, -7.4353,  4.3916]),
 tensor([4.7906]))

In [12]:
def synthetic_data(true_w, true_b, num_examples):
    dim_features = true_w.shape[0]
    X = torch.normal(0, 1, (num_examples, dim_features))
    y = torch.mv(X, true_w) + true_b
    y += torch.normal(0, 0.01, (num_examples, ))
    return X, y

num_examples = 400
features, labels = synthetic_data(true_w, true_b, num_examples)
features.shape, labels.shape

(torch.Size([400, 8]), torch.Size([400]))

In [13]:
def data_iter(features, labels, batch_size):
    num_examples = features.shape[0]
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = indices[i: i + batch_size]
        yield features[batch_indices], labels[batch_indices]

batch_size = 20

In [14]:
for X, Y in data_iter(features, labels, batch_size):
    print(X.shape, Y.shape)
    break

torch.Size([20, 8]) torch.Size([20])


### 定义模型与参数

In [15]:
def linreg(w, b, X):
    return torch.mv(X, w) + b

w = torch.normal(1, 0.1, (dim_features, ), requires_grad=True)
b = torch.normal(0, 0.1, (1, ), requires_grad=True)

### 定义损失函数与优化算法

In [16]:
def squared_loss(y_hat, y):
    loss = ((y_hat - y) ** 2) / 2
    return loss.sum() / y_hat.shape[0]

In [17]:
def sgd(params, lr):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad
            param.grad.zero_()

### 训练模型并检验正确性

In [18]:
lr = 0.05
num_epochs = 10
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y, in data_iter(features, labels, batch_size):
        l = loss(net(w, b, X), y)
        l.backward()
        sgd([w, b], lr)
    with torch.no_grad():
        l = loss(net(w, b, features), labels)
        print(f'epoch {epoch + 1}: loss = {float(l):f}')
        
w - true_w, b - true_b

epoch 1: loss = 7.802693
epoch 2: loss = 0.880278
epoch 3: loss = 0.106145
epoch 4: loss = 0.013621
epoch 5: loss = 0.001870
epoch 6: loss = 0.000301
epoch 7: loss = 0.000086
epoch 8: loss = 0.000055
epoch 9: loss = 0.000050
epoch 10: loss = 0.000049


(tensor([-0.0008,  0.0004,  0.0005,  0.0007,  0.0001,  0.0005,  0.0004, -0.0005],
        grad_fn=<SubBackward0>),
 tensor([0.0004], grad_fn=<SubBackward0>))